# ICCD753 Recuperación de Información 2025-B
# Examen Bimestral - Sistema Básico de Recuperación de Información
### Base de datos: Rotten Tomatoes movies and critic reviews
## Josune Singaña

## PARTE 1: IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN INICIAL

In [7]:
import numpy as np
import pandas as pd
import re
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from numpy.linalg import norm

# --- CONFIGURACIÓN INICIAL ---
def inicializar_recursos():
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords', quiet=True)
    
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt', quiet=True)

# Inicializamos al importar la librería
inicializar_recursos()
stop_words = set(stopwords.words('english'))

## PARTE 2: CARGA Y PREPROCESAMIENTO DE DATOS

In [8]:
# Cargar los datasets
df_movies = pd.read_csv('/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_movies.csv')
df_reviews = pd.read_csv('/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_critic_reviews.csv')

print(f"\n Películas cargadas: {len(df_movies)} registros")
print(f" Reviews cargadas: {len(df_reviews)} registros")

# Explorar estructura de datos
print("\n--- Columnas de Movies ---")
print(df_movies.columns.tolist())
print("\n--- Columnas de Reviews ---")
print(df_reviews.columns.tolist())

# Vista previa
print("\n--- Muestra de Películas ---")
print(df_movies.head(3))


 Películas cargadas: 17712 registros
 Reviews cargadas: 1130017 registros

--- Columnas de Movies ---
['rotten_tomatoes_link', 'movie_title', 'movie_info', 'critics_consensus', 'content_rating', 'genres', 'directors', 'authors', 'actors', 'original_release_date', 'streaming_release_date', 'runtime', 'production_company', 'tomatometer_status', 'tomatometer_rating', 'tomatometer_count', 'audience_status', 'audience_rating', 'audience_count', 'tomatometer_top_critics_count', 'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count']

--- Columnas de Reviews ---
['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name', 'review_type', 'review_score', 'review_date', 'review_content']

--- Muestra de Películas ---
  rotten_tomatoes_link                                        movie_title  \
0            m/0814255  Percy Jackson & the Olympians: The Lightning T...   
1            m/0878835                                        Please Give   
2                 m/10   

In [9]:
stop_words = set([
    'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for',
    'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'were', 'been',
    'be', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could',
    'should', 'may', 'might', 'can', 'this', 'that', 'these', 'those', 'i',
    'you', 'he', 'she', 'it', 'we', 'they', 'what', 'which', 'who', 'when',
    'where', 'why', 'how', 'all', 'each', 'every', 'both', 'few', 'more',
    'most', 'other', 'some', 'such', 'than', 'too', 'very', 'just', 'its'
])

# --- FUNCIONES DE PREPROCESAMIENTO ---
def procesar_texto_seguro(texto):
    # Limpieza robusta: Minúsculas -> Sin HTML -> Sin signos -> Tokenización -> Stopwords"""
    if not isinstance(texto, str): 
        return []
    
    text = texto.lower()
    text = re.sub(pattern=r"<.*?>", repl='', string=text)
    text = re.sub(r"[^a-z0-9\s]", '', text) 
    text = re.sub(r'\s+', ' ', text).strip()
    
    tokens = text.split()
    tokens_filtrados = [t for t in tokens if t not in stop_words and len(t) > 2]
    
    return tokens_filtrados

# 2.1 Limpiar datos de películas
print("\n2.1 Limpia datos de películas")

# Identificar columnas relevantes (ajustar según tu dataset)
columnas_texto = []
for col in df_movies.columns:
    if any(keyword in col.lower() for keyword in ['title', 'genre', 'synopsis', 'description', 'movie_title']):
        columnas_texto.append(col)

print(f"Columnas identificadas para procesamiento: {columnas_texto}")

# Crear columna combinada de texto
df_movies['texto_combinado'] = ''
for col in columnas_texto:
    if col in df_movies.columns:
        df_movies['texto_combinado'] += ' ' + df_movies[col].fillna('').astype(str)

# 2.2 Agregar reviews de críticos
print("\n2.2 Agrega reviews de críticos...")

# Identificar columna de ID común
id_col_movies = None
id_col_reviews = None

for col in df_movies.columns:
    if 'id' in col.lower() or 'rotten_tomatoes' in col.lower():
        id_col_movies = col
        break

for col in df_reviews.columns:
    if 'id' in col.lower() or 'rotten_tomatoes' in col.lower():
        id_col_reviews = col
        break

print(f"ID Movies: {id_col_movies}, ID Reviews: {id_col_reviews}")

# Agrupar reviews por película
if id_col_reviews:
    review_text_col = [col for col in df_reviews.columns if 'review' in col.lower() or 'content' in col.lower()][0]
    reviews_agrupadas = df_reviews.groupby(id_col_reviews)[review_text_col].apply(
        lambda x: ' '.join(x.fillna('').astype(str))
    ).reset_index()
    reviews_agrupadas.columns = [id_col_movies, 'reviews_texto']
    
    # Merge con películas
    df_movies = df_movies.merge(reviews_agrupadas, on=id_col_movies, how='left')
    df_movies['texto_combinado'] += ' ' + df_movies['reviews_texto'].fillna('')

print(f"\nTexto combinado creado para {len(df_movies)} películas")

# 2.3 Tokenización del corpus
print("\n2.3 Tokenizando corpus...")
corpus_tokens = df_movies['texto_combinado'].apply(procesar_texto_seguro).tolist()

# Estadísticas de preprocesamiento
tokens_totales = sum(len(doc) for doc in corpus_tokens)
tokens_unicos = len(set(token for doc in corpus_tokens for token in doc))

print(f"\n--- ESTADÍSTICAS DE PREPROCESAMIENTO ---")
print(f"Documentos procesados: {len(corpus_tokens)}")
print(f"Tokens totales: {tokens_totales:,}")
print(f"Vocabulario único: {tokens_unicos:,}")
print(f"Promedio tokens/doc: {tokens_totales/len(corpus_tokens):.1f}")


2.1 Limpia datos de películas
Columnas identificadas para procesamiento: ['movie_title', 'genres']

2.2 Agrega reviews de críticos...
ID Movies: rotten_tomatoes_link, ID Reviews: rotten_tomatoes_link

Texto combinado creado para 17712 películas

2.3 Tokenizando corpus...

--- ESTADÍSTICAS DE PREPROCESAMIENTO ---
Documentos procesados: 17712
Tokens totales: 1,228,506
Vocabulario único: 12,919
Promedio tokens/doc: 69.4


##  SECCIÓN 3: CONSTRUCCIÓN DEL SISTEMA

In [10]:
def obtener_vocabulario(corpus_tokens):
   # Genera vocabulario único ordenado.
    vocabulario = set()
    for doc in corpus_tokens:
        vocabulario.update(doc)
    return sorted(list(vocabulario))

def calcular_idf(corpus, vocabulario):
    """Calcula IDF para cada término del vocabulario."""
    N = len(corpus)
    idf_dict = {}
    
    df_dict = dict.fromkeys(vocabulario, 0)
    for doc in corpus:
        for palabra in set(doc):
            if palabra in df_dict:
                df_dict[palabra] += 1
                
    for palabra, df in df_dict.items():
        idf_dict[palabra] = math.log10(N / (df + 1))
        
    return idf_dict

def calcular_tf(documento):
    """Calcula TF para un documento."""
    tf_dict = {}
    largo_doc = len(documento)
    conteo = Counter(documento)
    
    for palabra, count in conteo.items():
        if largo_doc > 0:
            tf_dict[palabra] = count / largo_doc
        else:
            tf_dict[palabra] = 0
    return tf_dict

def generar_matriz_tfidf(corpus):
    """Genera matriz TF-IDF completa."""
    vocab = obtener_vocabulario(corpus)
    idf_dict = calcular_idf(corpus, vocab)
    matriz_datos = []
    
    for doc in corpus:
        tf_dict = calcular_tf(doc)
        fila_vector = []
        for palabra in vocab:
            valor_tf = tf_dict.get(palabra, 0)
            valor_idf = idf_dict[palabra]
            fila_vector.append(valor_tf * valor_idf)
        matriz_datos.append(fila_vector)
        
    return pd.DataFrame(matriz_datos, columns=vocab)

# Construir sistema
print("\n3.1 Generando vocabulario...")
vocabulario = obtener_vocabulario(corpus_tokens)
print(f"✓Vocabulario: {len(vocabulario)} términos")

print("\n3.2 Calculando IDF...")
idf_dict = calcular_idf(corpus_tokens, vocabulario)
print(f" IDF calculado para {len(idf_dict)} términos")

print("\n3.3 Generando matriz TF-IDF...")
df_tfidf = generar_matriz_tfidf(corpus_tokens)
print(f" Matriz TF-IDF: {df_tfidf.shape[0]} docs x {df_tfidf.shape[1]} términos")

# Análisis de términos más importantes
print("\n--- TÉRMINOS CON MAYOR IDF (más discriminativos) ---")
top_idf = sorted(idf_dict.items(), key=lambda x: x[1], reverse=True)[:15]
for term, idf_val in top_idf:
    print(f"{term:20s}: {idf_val:.4f}")


3.1 Generando vocabulario...
✓Vocabulario: 12919 términos

3.2 Calculando IDF...
 IDF calculado para 12919 términos

3.3 Generando matriz TF-IDF...
 Matriz TF-IDF: 17712 docs x 12919 términos

--- TÉRMINOS CON MAYOR IDF (más discriminativos) ---
10000               : 3.9472
1000000             : 3.9472
1001                : 3.9472
100yearold          : 3.9472
102                 : 3.9472
104210861088        : 3.9472
1080                : 3.9472
109                 : 3.9472
10x10               : 3.9472
110th               : 3.9472
111111              : 3.9472
1114                : 3.9472
112                 : 3.9472
1138                : 3.9472
1155                : 3.9472


## 4: FUNCIONES DE BÚSQUEDA

In [11]:
def vectorizar_consulta(consulta, vocabulario, idf_dict):
    #Vectoriza consulta usando TF-IDF."""
    tokens = procesar_texto_seguro(consulta)
    
    tf_val = 1 / len(tokens) if len(tokens) > 0 else 0
    tf_query = {token: tf_val for token in tokens}
    
    vector_consulta = []
    for palabra in vocabulario:
        tf = tf_query.get(palabra, 0)
        idf = idf_dict.get(palabra, 0)
        vector_consulta.append(tf * idf)
        
    return np.array(vector_consulta)

def calcular_similitud_coseno(vector_query, matriz_tfidf):
    #Calcula similitud coseno entre consulta y documentos."""
    ranking = []
    norma_q = norm(vector_query)
    
    if norma_q == 0: 
        return []
        
    for doc_idx, row in matriz_tfidf.iterrows():
        vector_doc = np.array(row)
        dot_product = np.dot(vector_query, vector_doc)
        norma_d = norm(vector_doc)
        
        if norma_d > 0:
            similitud = dot_product / (norma_q * norma_d)
        else:
            similitud = 0
        ranking.append((doc_idx, similitud))
        
    return ranking

def buscar_documentos(consulta, df_tfidf, vocabulario, idf_dict, top_n=5):
    #Busca documentos relevantes para una consulta."""
    print(f"\nBuscando: '{consulta}'")
    print("-" * 70)
    
    vec_q = vectorizar_consulta(consulta, vocabulario, idf_dict)
    scores = calcular_similitud_coseno(vec_q, df_tfidf)
    ranking_ordenado = sorted(scores, key=lambda x: x[1], reverse=True)
    
    resultados_finales = []
    for doc_id, score in ranking_ordenado[:top_n]:
        if score > 0:
            resultados_finales.append({
                'Documento_ID': doc_id,
                'Relevancia': round(score, 4)
            })
    
    return pd.DataFrame(resultados_finales)


## 5: SIMULACIONES DE CONSULTA

In [12]:
# Obtener columna de título
titulo_col = [col for col in df_movies.columns if 'title' in col.lower()][0]

# Consultas del examen
consultas = [
    "Películas sobre viajes espaciales",
    "Películas para ver en familia",
    "Películas de acción y aventura",
    "Comedias románticas",
    "Películas de terror y suspenso"
]

resultados_todas_consultas = {}

for consulta in consultas:
    # Buscar
    df_resultados = buscar_documentos(consulta, df_tfidf, vocabulario, idf_dict, top_n=10)
    
    if len(df_resultados) > 0:
        # Agregar información de películas
        df_resultados['Título'] = df_resultados['Documento_ID'].apply(
            lambda x: df_movies.iloc[x][titulo_col]
        )
        
        # Reordenar columnas
        df_resultados = df_resultados[['Documento_ID', 'Título', 'Relevancia']]
        
        print(f"\n{df_resultados.to_string(index=False)}")
        resultados_todas_consultas[consulta] = df_resultados
    else:
        print("No se encontraron resultados relevantes.")
        resultados_todas_consultas[consulta] = pd.DataFrame()


Buscando: 'Películas sobre viajes espaciales'
----------------------------------------------------------------------

 Documento_ID                                    Título  Relevancia
         2512 All About My Mother (Todo sobre mi madre)      0.5017

Buscando: 'Películas para ver en familia'
----------------------------------------------------------------------

 Documento_ID                                                                 Título  Relevancia
         4174                                              Busco Novio Para Mi Mujer      0.4817
        17187 What Have I Done to Deserve This? (Qu he hecho yo para merecer esto!!)      0.3851
         9528                    Like Water for Chocolate (Como Agua para Chocolate)      0.3507

Buscando: 'Películas de acción y aventura'
----------------------------------------------------------------------
No se encontraron resultados relevantes.

Buscando: 'Comedias románticas'
-----------------------------------------------------

## 6: ANÁLISIS DE RESULTADOS

In [13]:
print("\n ANÁLISIS CUANTITATIVO:")


for consulta, df_res in resultados_todas_consultas.items():
    if len(df_res) > 0:
        avg_score = df_res['Relevancia'].mean()
        max_score = df_res['Relevancia'].max()
        min_score = df_res['Relevancia'].min()
        
        print(f"\nConsulta: {consulta}")
        print(f"  • Resultados encontrados: {len(df_res)}")
        print(f"  • Relevancia promedio: {avg_score:.4f}")
        print(f"  • Relevancia máxima: {max_score:.4f}")
        print(f"  • Relevancia mínima: {min_score:.4f}")


 ANÁLISIS CUANTITATIVO:

Consulta: Películas sobre viajes espaciales
  • Resultados encontrados: 1
  • Relevancia promedio: 0.5017
  • Relevancia máxima: 0.5017
  • Relevancia mínima: 0.5017

Consulta: Películas para ver en familia
  • Resultados encontrados: 3
  • Relevancia promedio: 0.4058
  • Relevancia máxima: 0.4817
  • Relevancia mínima: 0.3507

Consulta: Películas de terror y suspenso
  • Resultados encontrados: 10
  • Relevancia promedio: 0.6365
  • Relevancia máxima: 0.7622
  • Relevancia mínima: 0.5507


LIMITACIONES IDENTIFICADAS:
   - El sistema es puramente léxico (no considera sinónimos)
   -  No captura relaciones semánticas entre palabras
- Sensible a la calidad de la metadata disponible